In [26]:
import os
import requests
from bs4 import BeautifulSoup as BS
from pprint import pprint

def check_tag(tag, subtag, check):
    try:
        if check in tag[subtag]:
            return True
        return False
    except KeyError:
        return False
        
ap_poll = {}
total_schools = {}
school_appear = {}
school_trans = {'Brigham Young': 'BYU',
                'Southern Cal': 'Southern California',
                'Southern Miss': 'Southern Mississippi',
                'Miami-Florida': 'Miami',
                'Nebraksa': 'Nebraska',
                'Texas A&M': 'Texas A&M;',
                'Louisiana St.': 'LSU',
                'Miami-Ohio': 'Miami (OH)'}

base_url = "http://preseason.stassen.com/final-ap-poll/%s.html"

for y in range(2003, 2013):
    comb_year = y+1
    ap_poll[comb_year] = []
    year = str(y)
    year_url = base_url % year
    response = requests.get(year_url)
    html = response.text.encode('utf-8').__str__()
    n = html.find("<html")
    year_soup = BS(html[n:], "html.parser")
    table_list = year_soup.find_all('table')
    for table in table_list:
        trs = table.find_all('tr')
        for tr in trs:
            if check_tag(tr, 'align', 'CENTER'):
                team = tr.contents[3].contents[0]
                if '(' in team:
                    start = team.find('(')
                    team = team[:start - 1]
                    if team in school_trans:
                        team = school_trans[team]
                if team != 'Team':
                    if team in school_trans:
                        team = school_trans[team]
                    if team not in total_schools:
                        total_schools[team] = 0
                    if team not in school_appear:
                        school_appear[team] = 0
                    school_appear[team] += 10
                    rank = tr.contents[1].contents[0]
                    points = tr.contents[7].contents[0].replace(',','')
                    ap_poll[comb_year].append([team, rank, points])

outd = open('OUTPUT_DRAFT.csv').read().split('\n')

point = outd[0].find('POS')
final_str = outd[0][:point] + 'AP Poll Points,' + outd[0][point:] + '\n'

for o in outd[1:-1]:
    os = o.split(',')
    school = os[2]
    year = int(os[0])
    ap_year = ap_poll[year]
    found = False
    for s in range(len(ap_year)):
        ss = ap_year[s]
        if school == ss[0]:
            found = True
            points = str(int(ss[2]) / 100)
            total_schools[ss[0]] += 1
    if not found:
        try:
            points = str(school_appear[school] / 100)
        except KeyError:
            points = '0'
    new_list = os[:3] + [points] + os[3:]
    for i in new_list:
        final_str += i + ','
    final_str += '\n'

outdp = open('OUTPUT_DRAFT2.csv','w')
outdp.write(final_str)
outdp.close()

print('Done')

Done
